# Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Preprocessing

In [ ]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [ ]:
n_samples, n_features = X.shape
print(X.shape)

(569, 30)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# Model
## f = wx + b, sigmoid at the end

In [ ]:
class LogisticRegression(nn.Module):

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

In [ ]:
learning_rate = 0.02
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 200

for epoch in range(num_epochs):
  # forward pass and loss
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  # backward pass
  loss.backward()

  # updates
  optimizer.step()

  # zero gradients
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'Epoch: {epoch+1}, loss={loss.item():.4f}')


with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'Accuracy = {acc:.4f}')

Epoch: 10, loss=0.1358
Epoch: 20, loss=0.1340
Epoch: 30, loss=0.1322
Epoch: 40, loss=0.1305
Epoch: 50, loss=0.1289
Epoch: 60, loss=0.1274
Epoch: 70, loss=0.1260
Epoch: 80, loss=0.1246
Epoch: 90, loss=0.1233
Epoch: 100, loss=0.1221
Epoch: 110, loss=0.1209
Epoch: 120, loss=0.1198
Epoch: 130, loss=0.1187
Epoch: 140, loss=0.1176
Epoch: 150, loss=0.1166
Epoch: 160, loss=0.1156
Epoch: 170, loss=0.1147
Epoch: 180, loss=0.1138
Epoch: 190, loss=0.1130
Epoch: 200, loss=0.1121
Accuracy = 0.9912
